Age Demographics Across the US

In [50]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.dataframe import dataframe_to_rows


In [51]:
api_key = 'f0c4eb00eca42436fb48ac7c95e9b0c902c5eeb4'

def get_data(year):
    url = f'https://api.census.gov/data/{year}/acs/acs5/subject?'

    params = {
        'get': 'NAME,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,S0101_C01_010E,S0101_C01_011E,S0101_C01_012E,S0101_C01_013E,S0101_C01_014E,S0101_C01_015E,S0101_C01_016E,S0101_C01_017E,S0101_C01_018E,S0101_C01_019E',
        'for': 'state:*',
        'key': api_key
    }

    response = requests.get(url, params=params)
    
    data = response.json()
    
    return data
    
    

In [52]:
year = '2022'
json_data = get_data(year)

df = pd.DataFrame(json_data[1:], columns=json_data[0])

df = df.rename(columns={
    'S0101_C01_001E': 'Total Population',
    'S0101_C01_002E': 'Under 5',
    'S0101_C01_003E': '5-9',
    'S0101_C01_004E': '10-14',
    'S0101_C01_005E': '15-19',
    'S0101_C01_006E': '20-24',
    'S0101_C01_007E': '25-29',
    'S0101_C01_008E': '30-34',
    'S0101_C01_009E': '35-39',
    'S0101_C01_010E': '40-44',
    'S0101_C01_011E': '45-49',
    'S0101_C01_012E': '50-54',
    'S0101_C01_013E': '55-59',
    'S0101_C01_014E': '60-64',
    'S0101_C01_015E': '65-69',
    'S0101_C01_016E': '70-74',
    'S0101_C01_017E': '75-79',
    'S0101_C01_018E': '80-84',
    'S0101_C01_019E': '85+'
})



In [53]:
# Convert columns to numeric
for col in df.columns[1:]:
    df[col] = pd.to_numeric(df[col])


In [54]:
# Calculate percentages
age_groups = df.columns[2:20]
for col in age_groups:
    df[f'{col}_pct'] = (df[col] / df['Total Population']) * 100
   
pct_cols = [f'{col}_pct' for col in age_groups]




In [59]:
colors = ['blue', 'red', 'green', 'yellow', 'pink', 'purple', 'black', 'cyan', 'orange', 'brown', 'gray', 'cyan', 'olive']

df.set_index('NAME')[pct_cols].plot(kind='bar', stacked=True, figsize=(14, 8), color=colors)
plt.title('Age Demographics Across States in 2022')
plt.xlabel('State')
plt.ylabel('Percentage of Population')
plt.legend(age_groups[:18], title='Age Groups', bbox_to_anchor=(1.05, 1), loc='upper left')


plt.tight_layout()
plot_file = 'my_acs_plot.png'
plt.savefig(plot_file, dpi=150)
plt.close()

In [60]:
file_name = 'ACS_data.xlsx'

# Save data to an excel file
writer = pd.ExcelWriter(file_name, engine='openpyxl')
df.to_excel(writer, sheet_name='ACS Data', index=False)

wb = writer.book

# Load image to new sheet
ws = wb.create_sheet(title='Plot')
img = Image(plot_file)
ws.add_image(img, 'A1')

wb.save(file_name)
plt.show()